# Estudo de memória de massa
Cliente: Mineração Serra Morena LTDA (10.754.267/0001-28 / UC 6/1704190-6)  
Período de análise: 01/09/2021 00:15 a 01/12/2021 00:00

In [ ]:
k_med = 700.0
# print(f"   >>> k_med = {k_med}")

## Memória de massa de demanda
Arquivo CSV ("dem_02520700_20220112_113243.csv") : 

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [ ]:
dem_filename = 'dem_02520700_20220112_113243.csv'
df_dem = pd.read_csv(dem_filename, delimiter=';')

# rename columns
df_dem.rename(columns = {'Data': 'data', 'kW fornecido': 'kw', 'Postos horários': 'posto', 'UFER': 'ufer', 'DMCR': 'dmcr'}, inplace = True)
df_dem['posto'] = list(map(lambda s: s.replace('Fora Ponta','FP')
                                       .replace('Ponta', 'PT'),
                           df_dem['posto']))
df_dem['data'] = pd.to_datetime(df_dem['data'], format='%d/%m/%Y %H:%M')              # convert to timedate
df_dem.set_index('data')                                                              # set the index
df_dem['kw'], df_dem['ufer'], df_dem['dmcr'] = \
    map(lambda s: s.str.replace(',', '.')
                   .str.replace('-', '0')
                   .astype('float64'),
        (df_dem['kw'], df_dem['ufer'], df_dem['dmcr']))                              # convert to float
df_dem['kw'], df_dem['ufer'], df_dem['dmcr'] = map(lambda x: x*k_med, 
                                                   (df_dem['kw'], df_dem['ufer'], df_dem['dmcr']))
df_dem.drop('Dia', axis='columns', inplace=True)                                     # remove unnecessary columns

# print('Primeiras 5 linhas da tabela:')
# display(df_dem.head())
# print('Últimas5 linhas da tabela:')
# display(df_dem.tail())

## Memória de massa de consumo
Arquivo CSV ("dem_02520700_20220112_113243.csv") : 

In [ ]:
cons_filename = 'cons_02520700_20220112_112952.csv'
df_cons = pd.read_csv(cons_filename, delimiter=';')

# rename columns
df_cons.rename(columns = {'Data': 'data', 'Posto': 'posto', 'kWh': 'kwh'}, inplace = True)
df_cons['posto'] = list(map(lambda s: s.replace('Fora Ponta','FP')
                                       .replace('Ponta', 'PT'),
                           df_cons['posto']))
df_cons['data'] = pd.to_datetime(df_cons['data'], format='%d/%m/%Y %H:%M')       # convert to timedate
df_cons['kwh'] = list(map(lambda s: float(s.replace(',', '.')
                                     .replace('-', '0')),
                        (df_cons['kwh'])))                                       # convert to float
df_cons['kwh'] = list(map(lambda x: x*k_med, (df_cons['kwh'])))
df_cons.drop('Dia', axis='columns', inplace=True)                                # remove unnecessary data and set index

# print('Primeiras 5 linhas da tabela:')
# display(df_cons.head())
# print('Últimas5 linhas da tabela:')
# display(df_cons.tail())

In [ ]:
# joined CONS and DEM dataframe 
df = pd.concat([df_dem, df_cons['kwh']], axis=1)                                           # merge the two dataframes
df['hr_int'] = (df.index + 2) // 4                                                         # group each 4 rows
df = df[['hr_int', 'data', 'posto', 'kwh', 'kw', 'ufer', 'dmcr']]                          # only/reorderes columns
# display(df.head())

In [ ]:
# Dataframe for UFER and DMCR analisys
df_ufer = df.groupby(['hr_int', 'posto']).agg({'kwh': 'sum', 'kw': 'sum', 'ufer':'sum', 'dmcr': 'sum', 'data': 'max'}).reset_index()
df_ufer = df_ufer[['data', 'posto', 'kwh', 'ufer', 'dmcr']]

# print('Primeiras 5 linhas da tabela:')
# display(df_ufer.head())
# print('Últimas5 linhas da tabela:')
# display(df_ufer.tail())

In [ ]:
import numpy as np

In [ ]:
P1 = df_ufer['kwh']
UFER = df_ufer['ufer']
DMCR = df_ufer['dmcr']
Sr = (P1 + UFER)/0.92
Qp = P1 * np.tan(np.arccos(0.92))
Ft = P1/Sr
Qt = Sr * np.sin(np.arccos(Ft))
Qe = Qt - Qp
df_ufer['ft'] = round(Ft,2)
df_ufer['qe'] = round(Qe)
df_ufer.fillna(0, inplace=True)

# print('Primeiras 5 linhas da tabela:')
# display(df_ufer.head())
# print('Últimas5 linhas da tabela:')
# display(df_ufer.tail())

## Gráfico e tabelas

In [ ]:
# plot the dat
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as wdg
from IPython.display import clear_output
from ipywidgets import VBox, HBox, Label, Layout, Output
from datetime import date, datetime
import calendar

In [ ]:
months = df_ufer['data'].dt.strftime('%m/%Y').unique()
months_wdg = wdg.SelectMultiple(
                                    options=months,
                                    value=[],
                                    layout=Layout(width='95%')
                                )
weeks_wdg = wdg.SelectMultiple(layout=Layout(width='95%'))

table = wdg.Output(layout=Layout(height='300px'))
graph = wdg.Output()
    
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        if change['owner'] == months_wdg:
            first_month = datetime.strptime(months_wdg.value[0], '%m/%Y')
            last_month = datetime.strptime(months_wdg.value[-1], '%m/%Y')
            first_day = date(first_month.year, first_month.month, 1)
            last_day = date(last_month.year, last_month.month, calendar.monthrange(year=last_month.year, month=last_month.month)[1])
            weeks = range(first_day.isocalendar()[1], last_day.isocalendar()[1] + 1)
            weeks_wdg.options=weeks
            weeks_wdg.value=list(weeks)
            
        if change['owner'] == weeks_wdg:
            data = df_ufer[df_ufer['data'].dt.strftime('%U').isin(str(x) for x in weeks_wdg.value)]
            if data['qe'].empty:
                data['max_qe'] = list(map(lambda x: None, data['qe']))
            else:
                data['max_qe'] = max(data['qe'])
            data['qexc'] = list(map(lambda d: '' if d == 0 else d, data['qe']))
            data['ufer_fp'] = data['ufer'].where(data['posto'] == 'FP').fillna(0)
            data['ufer_pt'] = data['ufer'].where(data['posto'] == 'PT').fillna(0)
            with graph:
                clear_output(True)
                fig = go.Figure()
                fig.add_trace(
                    go.Bar(
                            x=data['data'],
                            y=data['ufer_fp'],
                            marker=dict(color = 'blue'),
                            legendgroup='ufer',
                            legendgrouptitle_text='UFER',
                            name='Fora de Ponta'
                    )
                )
                fig.add_trace(
                    go.Bar(
                            x=data['data'],
                            y=data['ufer_pt'],
                            marker=dict(color = 'green'),
                            legendgroup='ufer',
                            name='Ponta'
                    )
                )
                fig.add_trace(
                    go.Scatter(
                                x=data['data'],
                                y=data['qexc'],
                                mode='lines',
                                marker=dict(color = 'rgb(230,160,160)', size=2.5),
                                legendgroup='qe',
                                legendgrouptitle_text='Pot. reativa excedente*',
                                name='Por intervalo'
                    )
                )
                fig.add_trace(
                    go.Scatter(
                                x=data['data'],
                                y=data['max_qe'],
                                mode='lines',
                                marker=dict(color = 'red'),
                                legendgroup='qe',
                                name='Máxima do período'
                    )
                )
                fig.update_layout(
                                    legend_title_text='Legenda',
                                    title="UFER E POTÊNCIA REATIVA EXCEDENTE",
                                    xaxis_title="Data",
                                    yaxis_title="UFER / kVAR",
                                    font=dict(size=10)
                )
                fig.update_xaxes(tickangle=-90)
                fig.show()
                print('* O valor da potência reativa excedente é igual à potência de capacitores em deficiência')
            with table:
                clear_output(True)
                display(data[['data', 'posto', 'ufer', 'dmcr', 'ft', 'qe']])            

months_wdg.observe(on_change)
weeks_wdg.observe(on_change)

border = ''
display(HBox([
                VBox([
                        Label('Mês'), 
                        months_wdg,
                        Label('Semana do mês'),
                        weeks_wdg
                    ],
                    layout=Layout(display='flex', border=border, width='10%')
                ),
                VBox([
                        Label('Gráfico'),
                        graph
                    ],
                    layout=Layout(display='flex', border=border, width='60%')
                ),
                VBox([
                        Label('Tabela de dados'),
                        table
                    ],
                    layout=Layout(display='overflow', border=border, width='30%')
                )
            ])
)

## Resumo do excedente de reativos por mês

In [ ]:
df = df_ufer[['data', 'posto', 'qe']]
df = df.groupby([df_ufer['data'].dt.strftime('%m/%Y'), df_ufer['posto']]).agg({'qe': 'max'}).reset_index()
display(df)